<a href="https://colab.research.google.com/github/dellaanima/DLton_Zero_to_One/blob/main/finetune-hfmodel/00%2BAUG(A%2CB)_kogpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#  TPU 초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)



In [4]:
import pandas as pd # pandas 모듈 로드
import json  # json 모듈 로드
import re
import os
from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from collections import defaultdict, Counter

import tensorflow as tf
from transformers import TextClassificationPipeline
from transformers import BertTokenizer, TFBertForSequenceClassification   #BERT
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification   #RoBerta
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TFGPT2LMHeadModel, TFGPT2ForSequenceClassification, TFGPT2Model    #GPT2

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [158]:

data = pd.read_csv('/content/drive/MyDrive/dlton/data/train.csv')
data.head(20)

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...


In [159]:

data = pd.read_csv('/content/drive/MyDrive/dlton/data/TRAIN_FINAL_v1.csv')
data.head(20)

,Unnamed: 0,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...


데이터 추가

In [123]:
def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()

    return sentence


In [160]:

data['conversation'] = data['conversation'].apply(lambda x : preprocess_sentence(x))
data.head(20)

,Unnamed: 0,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...
1,1,협박 대화,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...
3,3,갈취 대화,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘 . 저녁 시로 . 가나다 음식점이요 . ? ...
6,6,기타 괴롭힘 대화,번 손님 아이스커피 두잔나왔습니다 아이스커피 ? 네 맛있게드세요 저기요 아가씨 나는...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요 ? 내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요 확인해보겠습니다 아까 저 시람은 할인 해주던데 네 저 손님은 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데 ? 그게 . 얼마나 해먹었어 ?...


In [161]:
from sklearn.preprocessing import LabelEncoder

# Your class mapping
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3
}

# Assuming you have initialized 'data' DataFrame and 'label_encoder' object

# Transform class labels using the mapping
data['class'] = data['class'].map(class_mapping)

In [162]:
data

,Unnamed: 0,class,conversation
0,0,0,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...
1,1,0,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,3,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...
3,3,1,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...
4,4,1,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...
...,...,...,...
4337,4337,1,안녕 학생 ! 뭐 ? 나 ? 네 이리 오세요 뭐라고 ? 왜요 . 그런 말이 있어요 ...
4338,4338,1,이봐 이리 오렴 . 내게 돈을 좀 주세요 . 나 ? 나 돈 없어 내가 맞으면 줄 거...
4339,4339,1,이봐 . 어 ? 나 ? 그래 너 . 왜 그래 얘야 ? 아저씨 제가 어린아이인가요 ?...
4340,4340,1,내가 학교에 있는 동안 게임 머니를 충전해줘 나도 학교에 간다고 ? 게임 머니 충전...


In [163]:
train_df= data

In [164]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [167]:
max_seq_len = 200

In [168]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [169]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [170]:
train_X, train_y = convert_examples_to_features(train_df['conversation'], train_df['class'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 4342/4342 [00:03<00:00, 1277.79it/s]


In [171]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [172]:
max_seq_len = 200

In [173]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [174]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [175]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [176]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [191]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2",
                                          num_labels = 4)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [192]:
model.fit(train_X, train_y, epochs=3, batch_size=32)

Epoch 1/3
136/136 [==============================] - 128s 356ms/step - loss: 1.0460 - accuracy: 0.6555
Epoch 2/3
136/136 [==============================] - 13s 93ms/step - loss: 0.3190 - accuracy: 0.8904
Epoch 3/3
136/136 [==============================] - 13s 94ms/step - loss: 0.2272 - accuracy: 0.9205


In [196]:
model.fit(train_X, train_y, epochs=3, batch_size=32)

Epoch 1/3
136/136 [==============================] - 13s 95ms/step - loss: 0.1146 - accuracy: 0.9622
Epoch 2/3
136/136 [==============================] - 13s 93ms/step - loss: 0.0853 - accuracy: 0.9701
Epoch 3/3
136/136 [==============================] - 13s 93ms/step - loss: 0.0553 - accuracy: 0.9804


In [178]:
model.fit(train_X, train_y, epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
123/123 [==============================] - 151s 494ms/step - loss: 0.9581 - accuracy: 0.6404 - val_loss: 1.1184 - val_accuracy: 0.4299
Epoch 2/3
123/123 [==============================] - 12s 100ms/step - loss: 0.3604 - accuracy: 0.8736 - val_loss: 1.6573 - val_accuracy: 0.3011
Epoch 3/3
123/123 [==============================] - 13s 106ms/step - loss: 0.1915 - accuracy: 0.9350 - val_loss: 2.5254 - val_accuracy: 0.2644


In [186]:
model.fit(train_X, train_y, epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
123/123 [==============================] - 13s 109ms/step - loss: 0.1248 - accuracy: 0.9606 - val_loss: 2.5321 - val_accuracy: 0.2621
Epoch 2/3
123/123 [==============================] - 13s 102ms/step - loss: 0.1241 - accuracy: 0.9539 - val_loss: 4.6262 - val_accuracy: 0.2230
Epoch 3/3
123/123 [==============================] - 13s 103ms/step - loss: 0.0765 - accuracy: 0.9713 - val_loss: 4.6696 - val_accuracy: 0.2414


In [187]:
# Load Test-set
test_file_path = '/content/drive/MyDrive/dlton/data/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)
test_dataset



,t_000,t_001,t_002,t_004,t_005,t_006,t_007,t_009,t_010,t_012,...,t_489,t_490,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
text,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...,그 사람 누구냐? 누구말하는거야? 다 알면서 모른척 하지마. 둘다 쏴버리기 전에...,야 저기야 닭꼬치 트럭왔다 응 그러네 그치? 너도 먹고 싶지? 어?나도? 그래 너 ...,야 너 2학년 김민석 맞지? 네 맞는데요. 혹시 누구신가요? 내가 누군지 궁금하면 ...,예전에 네가 나한테 했던 일 기억하지? 너도 당해봐 예전 일이라면 내가 사과할게. ...,김비서 지금 우리애 학교가서 집에다가 좀 데려다 주세요 사장님 그런 개인적인 일은 ...,...,너 어제 집에 바로 안 들어갔지? 어떻게 알았어? 내가 눈이 좋아 왜 묻는건데? 너...,철수야 말 들어야지 싫은데 철밥통 선생아 너 선생님한테 말버릇이 뭐야 야 내가 너말...,동생이쁘다. 나주라 안돼 우리동생건들지마. 왜 안되냐? 좀 달라고 니 목숨값으로 동...,혹시 나 오늘 2만원만 빌려주만 안돼? 응? 안돼. 나도 요즘 맨날 택시타느라 택시...,이봐 네? 누구세요? 입다물고 가진거 다내놔 경찰에 신고할거에요 할 수 있으면 해봐...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,엄마 저 그 돈 안해주시면 정말 큰일나요. 이유도 말하지 않고. 몇번째니 경민아....


In [180]:
test_data = test_dataset.transpose()

In [181]:
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [197]:
from copy import deepcopy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# Assuming you have initialized the 'tokenizer' and 'model' objects properly

test_ = deepcopy(test_data["text"])
test_ = pd.DataFrame({"text": test_})

input_ids = []
for example in tqdm(test_["text"]):
    bos_token = [tokenizer.bos_token]
    eos_token = [tokenizer.eos_token]
    tokens = bos_token + tokenizer.tokenize(example) + eos_token
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_ids.append(input_id)

max_seq_len = max(map(lambda x: len(x), input_ids))

# Make sure to set padding option and pad_token_id
input_ids = pad_sequences(input_ids, maxlen=max_seq_len,
                          value=0,  # You can replace 0 with your desired pad token id
                          padding='post')

tmp = model.predict(input_ids)
test_["class"] = tmp.tolist()
test_["label_class"] = test_["class"].apply(lambda x: np.argmax(x))
labels = pd.DataFrame(tmp.tolist()).add_prefix("label_")
labels.index = test_.index
test_ = pd.concat([test_, labels], axis=1)
Counter(test_["class"].apply(lambda x: np.argmax(x)))

100%|██████████| 400/400 [00:00<00:00, 1888.38it/s]


13/13 [==============================] - 1s 36ms/step


Counter({1: 108, 2: 112, 3: 100, 0: 80})

In [198]:
test_

,text,class,label_class,label_0,label_1,label_2,label_3
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,"[0.0011506063165143132, 0.9985659718513489, 3....",1,1.150606e-03,0.998566,3.329274e-06,2.801536e-04
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,"[4.309482548592314e-08, 1.1494819318613736e-06...",2,4.309483e-08,0.000001,9.999987e-01,1.030598e-07
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,"[0.00020732013217639178, 0.0038763696793466806...",2,2.073201e-04,0.003876,9.930184e-01,2.897835e-03
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,"[0.0009758688975125551, 0.3037336468696594, 1....",3,9.758689e-04,0.303734,1.402915e-05,6.952765e-01
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...,"[0.9915480017662048, 0.0026209179777652025, 2....",0,9.915480e-01,0.002621,2.119170e-06,5.828994e-03
...,...,...,...,...,...,...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,"[2.951260746897333e-08, 0.00041574257193133235...",2,2.951261e-08,0.000416,9.995690e-01,1.526204e-05
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,"[4.947704655933194e-05, 0.000665426894556731, ...",2,4.947705e-05,0.000665,9.991674e-01,1.177296e-04
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,"[2.5718352958392643e-07, 0.999997615814209, 5....",1,2.571835e-07,0.999998,5.395389e-08,2.160481e-06
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,"[0.00316192046739161, 3.0532970413332805e-05, ...",2,3.161920e-03,0.000031,9.962001e-01,6.074543e-04


In [200]:
# Save the 'label_class' column as a CSV file
label_class_column = test_["label_class"]
label_class_column.to_csv("./added_gpt2_4.csv", index=True)